# Experiments for image encoding evaluation

## Setup

### Google Drive setup

In [1]:
DRIVE_MOUNT_PATH = "/content/drive"
DRIVE_WORK_DIR = "MyDrive/Colab Notebooks/GEQIE"

try:
    from google.colab import drive
    print("Environment is Google Colab.")
    WORK_DIR = f"{DRIVE_MOUNT_PATH}/{DRIVE_WORK_DIR}"

    print(f"Mounting Google Drive '{WORK_DIR}'...")
    drive.mount(DRIVE_MOUNT_PATH)

    try:
        import geqie
        print("Dependencies already installed.")
    except:
        print("Installing dependencies...")
        ! uv pip install -e .
        ! uv pip install -r experiments/requirements/requirements.in
except Exception:
    print("Environment is not Google Colab... skipping.")

Environment is not Google Colab... skipping.


### Imports

In [2]:
import os
from pathlib import Path

In [3]:
from itertools import product

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageDraw
from tqdm import tqdm

import geqie
from geqie.encodings import frqi, ifrqi, mcqi, ncqi, neqr, qrci, qualpi

### Constants

In [4]:
CURRENT_PATH = Path(os.path.abspath('.'))

GRAYSCALE_PATH = CURRENT_PATH.parent / "experiments" / "generated_images" / "grayscale"
GRAYSCALE_METHODS = [
    frqi,
    neqr,
]


RGB_PATH = CURRENT_PATH.parent / "experiments" / "generated_images" / "rgb"
RGB_METHODS = [
    ifrqi,
    mcqi,
]

In [5]:
GRAYSCALE_IMAGE_PATHS = sorted(GRAYSCALE_PATH.glob("*.png"))
RGB_IMAGE_PATHS = sorted(RGB_PATH.glob("*.png"))

### Helpers

In [7]:
def plot_images_side_by_side(original, retrieved):
	image_original = Image.fromarray(original)
	draw_original = ImageDraw.Draw(image_original)
	image_retrieved = Image.fromarray(retrieved)
	draw_retrieved = ImageDraw.Draw(image_retrieved)
	display(image_original.resize((image_original.width*30, image_original.height*30), resample=Image.NEAREST))
	display(image_retrieved.resize((image_retrieved.width*30, image_retrieved.height*30), resample=Image.NEAREST))

## Experiment 1: Image Reconstruction Quality

In [10]:
for (method, image_path) in tqdm(list(product(GRAYSCALE_METHODS[:1], GRAYSCALE_IMAGE_PATHS))):
    image = Image.open(image_path)
    image = ImageOps.grayscale(image)
    image = np.asarray(image)
    print(image.shape)

    circuit = geqie.encode(method.init_function, method.data_function, method.map_function, image)
    result = geqie.simulate(circuit, 1024)
    retrieved_image = frqi.retrieve_function(result)

    print(f"Method: {method.__name__}, Image: {image_path.name}")
    plot_images_side_by_side(image, retrieved_image)

    # print(circuit.draw(output="text"))

  0%|          | 0/16 [00:00<?, ?it/s]

(8, 8)


  0%|          | 0/16 [27:47<?, ?it/s]


KeyboardInterrupt: 